In [50]:
import h5py
import numpy as np

In [6]:
file_path = 'Faller_et_al_2019_PNAS_EEG_Neurofeedback_VR_Flight/S01_F_CL_Sil_50_100.mat'

In [129]:
def extract_trials(filepath):
    with h5py.File(file_path, 'r') as f:
        raw_eeg_data = np.array(f['actualVariable']['EEG_full']['data']).T[:64]
        events = f['actualVariable']['EEG_full']['event']
        event_labels = []
        
        for i in range(len(events['type'])):
            event_type_ref = events['type'][i][0]
            event_latency_ref = events['latency'][i][0]
            
            event_type = ''.join(chr(x) for x in f[event_type_ref][:].flatten())
            event_latency = f[event_latency_ref][()][0]
            event_labels.append((event_type, event_latency))
    
        easy_data_list = []
        hard_data_list = []
        sampling_frequency = 256
        sample_duration = 30
        
        trial_starts = []
        trial_ends = []
        
        for event, time in event_labels:
            if event == "START-F_CL_Sil_50_100":
                trial_starts.append(time)
            elif event == "boundary":
                trial_ends.append(time)
        trial_ends.append(event_labels[-1][1])
        
        num_trials = len(trial_starts)
        for trial_idx in range(num_trials):
            start_time = trial_starts[trial_idx]
            end_time = trial_ends[trial_idx]
            start_sample = int(start_time)
            end_sample = int(end_time)
            
            trial_data = raw_eeg_data[:, start_sample:end_sample]
            trial_duration = end_time - start_time
            num_samples_in_trial = trial_data.shape[1]
            
            if trial_idx % 2 == 0:
                data_segment = trial_data[:, :sample_duration*sampling_frequency]
                easy_data_list.append(data_segment)
            else:
                data_segment = trial_data[:, -sample_duration*sampling_frequency:]
                hard_data_list.append(data_segment)
    
        easy_data_array = np.array(easy_data_list)  # Shape: (12, 64, 30*256)
        hard_data_array = np.array(hard_data_list)  # Shape: (12, 64, 30*256)
    return easy_data_array, hard_data_array

In [131]:
e, h = extract_trials(file_path)

/var/folders/1t/bjxpyljd2576zyjrjv5bnjq40000gn/T/ipykernel_20359/1830590696.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  start_sample = int(start_time)
/var/folders/1t/bjxpyljd2576zyjrjv5bnjq40000gn/T/ipykernel_20359/1830590696.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  end_sample = int(end_time)


In [145]:
e.shape, h.shape

((12, 64, 7680), (12, 64, 7680))